In [1]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import sys
import json
from numpy import linalg as LA
import tensorflow as tf
from sklearn.decomposition import PCA

from pathlib import Path
import dotenv
dotenv.load_dotenv()
p_code = Path(os.environ.get("HOME_DIR"))
p_data = Path(os.environ.get("DATA_DIR"))

net = 'stepnet'
PATH_YANGNET = os.path.join(p_code,net) 
sys.path.insert(0, PATH_YANGNET)

from task import generate_trials, rule_name, rule_index_map, rules_dict
from network import Model, get_perf, FixedPoint_Model
import tools
from analysis import clustering, standard_analysis, variance
import numpy.random as npr
from tools_lnd import get_T_inds, gen_trials_from_model_dir, gen_X_from_model_dir, make_lil_axes
from tools_lnd import find_closest_fp_loc, get_model_params


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# ##################################################################
# #fig save path
# ##################################################################
fig = 'fig5'
figpath = os.path.join(p_code,'figs',fig,fig+'_BCEFHIKLNOQR')
if not os.path.exists(figpath):
    os.makedirs(figpath)

In [3]:
# ##################################################################
# #Find right model dir
# ##################################################################
data_fldr = 'fig3'
file = 'LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1'
m = os.path.join(p_data,data_fldr,file)

In [5]:
lesion_folder = 'lesion_fps_hierarchical_ward_distance_opt_clust'
save_dir = os.path.join(m,lesion_folder)

cluster_var = np.load(os.path.join(save_dir,'cluster_var.npz'))
lesion_var = np.load(os.path.join(save_dir,'lesion_var.npz'))
perfs_changes = lesion_var['perfs_changes']

cluster_number = {}
cluster_number['Anti Stimulus'] = 5
cluster_number['Delayed Response'] = 2
cluster_number['Modality2'] = 10
cluster_number['Modality1'] = 6
cluster_names = ['Anti Stimulus','Delayed Response','Modality2','Modality1']

In [6]:
# def make_label_sets(rule_trains):
#     label_set = {}
#     label_set['anti'] = [('anti' in rule) or ('nogo' in rule) for rule in rule_trains]
#     label_set['delay'] = [('delay' in rule) or ('fd' in rule) for rule in rule_trains]
#     label_set['dm2'] = ['dm2' in rule for rule in rule_trains]
#     label_set['dm1'] = ['dm1' in rule for rule in rule_trains]
#     label_set['dmc'] = ['dmc' in rule for rule in rule_trains]
#     label_set['react'] = ~np.array(label_set['delay'])

#     label_reverse_set = {}
#     label_reverse_set['anti'] = ~np.array(label_set['anti'])
#     label_reverse_set['delay'] = ~np.array(label_set['delay'])
#     label_reverse_set['dm2'] = ['dm1' in rule for rule in rule_trains]
#     label_reverse_set['dm1'] = ['dm2' in rule for rule in rule_trains]
#     label_reverse_set['dmc'] = ~np.array(label_set['dmc'])
#     label_reverse_set['react'] = ~np.array(label_set['react'])
#     return label_set, label_reverse_set

In [7]:
# def make_label_sets(rule_trains):
    
#     rule_trains_random = np.random.permutation(rule_trains)
#     print(rule_trains_random)
    
#     label_set = {}
#     label_set['Anti'] = [('anti' in rule) for rule in rule_trains]
#     label_set['Delay'] = [('delay' in rule) or ('fd' in rule) for rule in rule_trains]
# #     label_set['Memory'] = [('delay' in rule) for rule in rule_trains]
#     label_set['Modality2'] = ['dm2' in rule for rule in rule_trains]
#     label_set['Modality1'] = ['dm1' in rule for rule in rule_trains]
#     label_set['Random2'] = ['dm2' in rule for rule in rule_trains_random]
#     label_set['Random3'] = ['anti' in rule for rule in rule_trains_random]
#     label_set['Random9'] = [('delay' in rule) or ('fd' in rule) for rule in rule_trains_random]

#     label_reverse_set = {}
#     label_reverse_set['Anti'] = ~np.array(label_set['Anti'])
#     label_reverse_set['Delay'] = ~np.array(label_set['Delay'])
# #     label_reverse_set['Memory'] = ~np.array(label_set['Memory'])
#     label_reverse_set['Modality2'] = ['dm1' in rule for rule in rule_trains]
#     label_reverse_set['Modality1'] = ['dm2' in rule for rule in rule_trains]
#     label_reverse_set['Random2'] = ['dm1' in rule for rule in rule_trains_random]
#     label_reverse_set['Random3'] = ~np.array(label_set['Random3'])
#     label_reverse_set['Random9'] = ~np.array(label_set['Random9'])
#     return label_set, label_reverse_set

In [8]:
# ##################################################################
# #Find right model dir
# ##################################################################

# rule_trains = ['fdgo', 'reactgo', 'delaygo', 'fdanti', 'reactanti', 'delayanti',
#           'delaydm1', 'delaydm2', 'contextdelaydm1', 'contextdelaydm2', 'multidelaydm',
#           'dmsgo', 'dmsnogo', 'dmcgo', 'dmcnogo']

# # rule_trains = ['fdgo', 'delaygo', 'fdanti']

# rule_trains_str = '_'.join(rule_trains)

# # parse input arguments as:
# rnn_type = 'LeakyRNN'
# activation = 'softplus'
# w_init = 'diag'
# ruleset = 'lr'
# n_tasks = str(len(rule_trains))
# n_rnn = str(128)
# l2w = float(-6)
# l2h = float(-6)
# l1w = float(0)
# l1h = float(0)
# lr = float(-6)
# seed = str(1)
# net_name = 'lr'+"{:.1f}".format(-lr)+'l2_w'+"{:.1f}".format(-l2w)+'_h'+"{:.1f}".format(-l2h)+'_'+rule_trains_str
# data_folder = 'data/rnn/multitask/stepnet/'

In [9]:
# # set of networks to compare
# rnn_type = 'LeakyRNN'
# activation = 'softplus'
# w_init = 'diag'
# data_folder ='final'
# color_set = {}
# color_set['lr'] = 'k'
# color_set['untrained'] = 'dodgerblue'
# color_set['no_noise'] = 'orangered'

# seed_set = [str(1),]
# max_d = 20
# n_rnn = str(128)
# x_ind_counter = 0
# ax1_xticks = []
# method = 'average'
# criterion = 'maxclust'
# all_epochs = ['stim1','stim2','delay1','delay2','go1']
# n_tasks = str(len(rule_trains))
# seed = 1
# lr = -7
# sigma_rec = 1/20
# sigma_x = 2/20
# ruleset = 'all'
# w_rec_coeff  = 8/10
# net_name = 'lr'+"{:.1f}".format(-lr)+'l2_w'+"{:.1f}".format(-l2w)+'_h'+"{:.1f}".format(-l2h)+'_'+rule_trains_str
# # data_folder_all = 'data/rnn/multitask/stepnet/'

# net_name = 'lr'+"{:.1f}".format(-lr)+'l2_w'+"{:.1f}".format(-l2w)+'_h'+"{:.1f}".format(-l2h)

# net_name2 = '_sig_rec'+str(sigma_rec)+'_sig_x'+str(sigma_x)+'_w_rec_coeff'+"{:.1f}".format(w_rec_coeff)+'_'+rule_trains_str

# m = os.path.join(p,'data','rnn','multitask',net,data_folder,ruleset,
#                         rnn_type,activation,w_init,str(len(rule_trains))+'_tasks',
#                         str(n_rnn)+'_n_rnn',net_name+net_name2,str(seed))

# net_name = 'lr'+"{:.1f}".format(-lr)+'l2_w'+"{:.1f}".format(-l2w)+'_h'+"{:.1f}".format(-l2h)+'_'+rule_trains_str
# m = os.path.join(p,'data','rnn','multitask',net,'final','all',rnn_type,activation,w_init,str(len(rule_trains))+'_tasks',str(n_rnn)+'_n_rnn',net_name,str(seed))

In [10]:
# rule_set_names = ['DelayPro', 'ReactPro', 'MemoryPro', 'DelayAnti', 'ReactAnti', 'MemoryAnti',
#               'IntegrationModality1', 'IntegrationModality2', 'ContextIntModality1', 'ContextIntModality2', 'IntegrationMultimodal',
#               'ReactMatch2Sample', 'ReactNonMatch2Sample', 'ReactCategoryPro', 'ReactCategoryAnti']

In [11]:
# rule_names = {'fdgo':'DelayPro', 
#               'reactgo':'ReactPro', 
#               'delaygo':'MemoryPro', 
#               'fdanti':'DelayAnti', 
#               'reactanti':'ReactAnti',
#               'delayanti':'MemoryAnti',
#               'delaydm1':'DelayD1', 
#               'delaydm2':'DelayD2', 
#               'contextdelaydm1':'ContextDelayD1', 
#               'contextdelaydm2':'ContextDelayD2', 
#               'multidelaydm':'MultiDelayD',
#               'dmsgo':'DM2SamplePro', 
#               'dmsnogo':'DM2SampleAnti', 
#               'dmcgo':'DM2CategoryPro', 
#               'dmcnogo':'DM2CategoryAnti'}

In [12]:
# label_set,label_reverse_set = make_label_sets(rule_trains)

In [13]:
# import matplotlib.patches as mpatches
# from scipy.cluster import hierarchy
# color_palette = ['b','c', 'm', 'y', 'r']
# color_palette = ['#377eb8', '#ff7f00', '#4daf4a',
#                   '#f781bf', '#a65628', '#984ea3',
#                  '#e41a1c', '#dede00']
# hierarchy.set_link_color_palette(color_palette)
# alphabet = {1:'a', 2:'b', 3:'c', 4:'d', 5:'e', 6:'f', 7:'g', 8:'h', 9:'i', 10:'j', 11:'k', 12:'l', 13:'m', 14:'n', 15:'o', 16:'p', 17:'q', 18:'r', 19:'s', 20:'t', 21:'u', 22:'v', 23:'w', 24:'x', 25:'y', 26:'z'}

In [14]:
# from network import get_perf
# from task import generate_trials
# from tools_lnd import take_names

# fontsize = 20
# tick_fontsize = fontsize*.75
# label_fontsize = fontsize*1
# plt.rcParams.update({'font.size': fontsize})

# def plot_lesion(m,task_list,epoch,lesion_units_list,cluster_label = [],lesion_num = []):

#     fig = plt.figure(figsize=(4, 4))
#     cmap = plt.get_cmap('hsv')
# #     c_full = cmap(0.1)
# #     c_lesion = cmap(.6)
#     c_full = 'dodgerblue'
#     c_lesion = 'r'#'brown'
#     al = .5
#     lw = 6
    
#     from tools_lnd import get_model_params
#     w_in, b_in, w_out, b_out = get_model_params(m)

#     D_out = {}
#     D_out['axes'] = w_out[:,1:]
#     D_out['labels'] = ['W_{out} cos(theta)','W_{out} sin(theta)']

#     model = FixedPoint_Model(m)
#     hp = model.hp
#     with tf.Session() as sess:
#         model.restore()
#         model._sigma=0
#         if len(lesion_units_list)>0:
#             model.lesion_units(sess, lesion_units_list)

#         for ri in range(len(task_list)):
#             rule = task_list[ri]
#             trial = generate_trials(rule, hp, 'test', noise_on = False)
#             feed_dict = tools.gen_feed_dict(model, trial, hp)
#             h_tf = sess.run(model.h, feed_dict=feed_dict) #(n_time, n_condition, n_neuron)
            
#             _components = 3
#             pca = PCA(n_components = n_components)
#             D_pca = {}
            
#             if epoch in trial.epochs.keys():
#                 T_inds = get_T_inds(trial,epoch) # grab epoch time indexing
#             else:
#                 T_inds = get_T_inds(trial,'go1')
                
#             h =  np.transpose(h_all_byrule[rule][T_inds,:,:],(2,1,0))# h_tf[:,range(1,n_trials),:],(2,1,0))
#             Xh = np.reshape(h,(h.shape[0],-1))
#             fp_pca = pca.fit_transform(Xh.T)
#             D_pca['axes'] = pca.components_.T
#             D_pca['labels'] = ['PCA '+str(x+1) for x in range(n_components)]

#             for trial_i in range(3,np.shape(trial.x)[1],int(np.shape(trial.x)[1]/20)): #includes mod2 trials
#                 c = cmap(trial.y_loc[-1,trial_i]/(2*np.pi))

# #                 ax = plt.subplot(len(task_list),2,1+(2*ri))
#                 ax = fig.add_axes([0,ri*.6,.3,.3]);
#                 X_rule = np.dot(h_all_byrule[rule][:,trial_i,:],D_pca['axes'])
#                 plt.plot(X_rule[:,0],X_rule[:,1],c = c_full,alpha = al,linewidth = lw)
#                 plt.plot(X_rule[-1,0],X_rule[-1,1],'^',c = c_full,alpha = al,linewidth = lw)

#                 X_dot = np.dot(h_tf[:,trial_i,:],D_pca['axes'])
#                 plt.plot(X_dot[:,0],X_dot[:,1],c = c_lesion,alpha = al,linewidth = lw)
#                 plt.plot(X_dot[-1,0],X_dot[-1,1],'^',c = c_lesion,alpha = al,linewidth = lw)
                
#                 _, rule_name, _, _ = take_names(epoch,rule)
#                 plt.title(cluster_label[ri]+r"$\bf{" + rule_name + "}$", y=.98, x = 1)

#                 plt.xlabel(D_pca['labels'][0])
#                 plt.ylabel(D_pca['labels'][1])
#                 ax.spines['right'].set_visible(False)
#                 ax.spines['top'].set_visible(False) 
#                 ax.spines['bottom'].set_visible(False)
#                 ax.spines['left'].set_visible(False)  
#                 ax.set_xticks([]) 
#                 ax.set_yticks([])

# #                 ax = plt.subplot(len(task_list),2,2+(2*ri))
#                 ax = fig.add_axes([.5,ri*.6,.3,.3]);
#                 X_rule = np.dot(h_all_byrule[rule][:,trial_i,:],D_out['axes'])
#                 plt.plot(X_rule[:,0],X_rule[:,1],c = c_full,alpha = al,linewidth = lw)
#                 plt.plot(X_rule[-1,0],X_rule[-1,1],'^',c = c_full,alpha = al,linewidth = lw)

#                 X_dot = np.dot(h_tf[:,trial_i,:],D_out['axes'])
#                 plt.plot(X_dot[:,0],X_dot[:,1],c = c_lesion,alpha = al,linewidth = lw)
#                 plt.plot(X_dot[-1,0],X_dot[-1,1],'^',c = c_full,alpha = al,linewidth = lw)

#                 plt.xlabel('OUT 1')
#                 plt.ylabel('OUT 2')
#                 ax.spines['right'].set_visible(False)
#                 ax.spines['top'].set_visible(False) 
#                 ax.spines['bottom'].set_visible(False)
#                 ax.spines['left'].set_visible(False)  
#                 ax.set_xticks([]) 
#                 ax.set_yticks([])
                

#     figname = 'viz_traj_'+'_'.join(task_list)+str(lesion_num)
#     plt.savefig(os.path.join(figpath,figname+'.pdf'),bbox_inches='tight')

In [15]:
# h_all_byepoch, h_all_byrule, _, _, _, _ = make_h_all(m,mode = 'test')

In [16]:
# # lesion cluster
# n_components = 2
# task_list = ['reactgo', 'delaygo']
# cluster = cluster_number['Delayed Response']+1
# lesion_units_list = cluster_var['lesion_units_list'][cluster]
# epoch = 'go1'
# plot_lesion(m,task_list,epoch,lesion_units_list,
#             cluster_label = ['Task w/o Delayed Response \n motif : ','Task w Delayed Response \n motif : '],
#             lesion_num = cluster)

In [17]:
# # lesion cluster
# task_list = ['delaygo', 'delayanti']
# cluster = cluster_number['Anti Stimulus']+1
# lesion_units_list = cluster_var['lesion_units_list'][cluster]
# epoch = 'stim1'
# plot_lesion(m,task_list,epoch,lesion_units_list,
#             cluster_label = ['Task w/o Anti Stimulus \n motif : ','Task w Anti Stimulus \n motif : '],lesion_num = cluster)

In [18]:
# # lesion cluster
# task_list = ['contextdelaydm1','contextdelaydm2']
# cluster = cluster_number['Modality2']+1
# lesion_units_list = cluster_var['lesion_units_list'][cluster]
# epoch = 'go1'
# plot_lesion(m,task_list,epoch,lesion_units_list,
#             cluster_label = ['Task w/o Modality 2 \n motif : ','Task w Modality 2 \n motif : '],lesion_num = cluster)

In [19]:
# # lesion cluster
# task_list = ['contextdelaydm2','contextdelaydm1']
# cluster = cluster_number['Modality1']+1
# lesion_units_list = cluster_var['lesion_units_list'][cluster]
# epoch = 'go1'
# plot_lesion(m,task_list,epoch,lesion_units_list,
#             cluster_label = ['Task w/o Modality 1 \n motif : ','Task w Modality 1 \n motif : '],lesion_num = cluster)

In [20]:
# from tools_lnd import remove_spines, get_xlim_diff, get_ylim_diff
# def make_lil_axes(ax,axes_labels,fontsize = 20,fac_len = 10):
    
#     remove_spines(ax)

#     x1,x2,x_diff = get_xlim_diff(ax)
#     y1,y2,y_diff = get_ylim_diff(ax)
    
#     if len(axes_labels)>2:
#         z1,z2,z_diff = get_zlim_diff(ax)
        
#         plt.plot([x1,x1+x_diff/fac_len],[y1,y1],[z1,z1],'-k')
#         plt.plot([x1,x1],[y1,y1+y_diff/fac_len],[z1,z1],'-k')
#         plt.plot([x1,x1],[y1,y1],[z1,z1+z_diff/fac_len],'-k')
        
#         ax.text(x1,y1-y_diff/8,z1,axes_labels[0],(1,0,0),
#             horizontalalignment='left',
#             verticalalignment='top',
#                  fontsize = fontsize)
#         ax.text(x1-x_diff/8,y1+y_diff/5,z1,axes_labels[1],(0,1,0),
#             horizontalalignment='left',
#             verticalalignment='bottom',
#                  fontsize = fontsize)
#         ax.text(x1-x_diff/15,y1,z1+z_diff/5,axes_labels[2],
#             horizontalalignment='right',
#             verticalalignment='bottom',
#                  fontsize = fontsize)
        
#         ax.set_zticks([])
        
#     else:
#         plt.plot([x1,x1+x_diff/10],[y1,y1],'-k')
#         plt.plot([x1,x1],[y1,y1+y_diff/10],'-k')

#         plt.text(x1,y1-y_diff/50,axes_labels[0],
#             horizontalalignment='left',
#             verticalalignment='top',
#                  fontsize = fontsize)
#         plt.text(x1-x_diff/18,y1,axes_labels[1], #/6
#             horizontalalignment='left',
#             verticalalignment='bottom',
#                  fontsize = fontsize,
#                  rotation = 90)
        
#     ax.set_yticks([])
#     ax.set_xticks([])

In [21]:
# for rule in ['contextdelaydm1','contextdelaydm2']:#
#     model = FixedPoint_Model(m)
#     with tf.Session() as sess:
#         model.restore()
#         model._sigma=0

# #         model.lesion_units(sess, lesion_units)
#         # get all connection weights and biases as tensorflow variables
#         var_list = model.var_list
#         # evaluate the parameters after training
#         params = [sess.run(var) for var in var_list]
#         # get hparams
#         hparams = model.hp
#         # create a trial
#         trial = generate_trials(rule, hparams, mode='test', noise_on=False, batch_size=40)# get feed_dict

In [22]:
# def make_ax_nice(ax, ax_labels, plot_zero_plane = True):
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['bottom'].set_visible(False)
#     ax.spines['left'].set_visible(False)
#     ax.set_yticks([])
#     ax.set_xticks([])
#     ax.set_zticks([])
#     epoch_name_ax, rule_name_ax, _, _ = take_names(ax_labels[0],ax_labels[1])
#     ax.set_xlabel(r"$\bf{"+rule_name_ax+"}$"+' \n '+epoch_name_ax+' State PC1', labelpad=0)
#     ax.set_ylabel(r"$\bf{"+rule_name_ax+"}$"+' \n '+epoch_name_ax+' State PC2', labelpad=0)
#     ax.set_zlabel(r"$\bf{"+rule_name_ax+"}$"+' \n '+epoch_name_ax+' State PC3', labelpad=0)
    
#     [x1,x2] = ax.get_xlim()
#     [y1,y2] = ax.get_ylim()
#     ax.set_xlim([x1+.1*x1,x2+.1*x2])
#     ax.set_ylim([y1+.1*y1,y2+.1*y2])
#     [x1,x2] = ax.get_xlim()
#     [y1,y2] = ax.get_ylim()
    
#     if plot_zero_plane:
#         xx, yy = np.meshgrid(np.linspace(x1, x2, num=2), np.linspace(y1, y2, num=2))
#         z = xx*0
#         ax.plot_surface(xx, yy, z, alpha=0.1)
#         ax.set_zlabel(r"$\bf{Output}$"+r'$\cos{\theta}$',labelpad=-10)
# #         ax.text(x1, y1, 0, 'Output Null', (1,0,0))
#     elif len(zlabel)>0:
#         ax.set_zlabel(zlabel,labelpad=-10)
#     ax.set_zlim([-1.1,1.1])
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_zticks([-1,1])

In [23]:
# def plot_N3D(ax, X, D, clist, linewidth = 6, alpha = .5, linestyle = '-'):
#     """Plot activity is some 2D space.

#         Args:
#             X: neural activity in Trials x Time x Neurons
#             D: Neurons x 2 plotting dims
#         """

#     cmap=plt.get_cmap('rainbow')
#     S = np.shape(X)[0]
    
#     for s in range(S):

#         if isinstance(clist, str) :
#             c = clist
#         elif len(clist)==1:
#             c = clist[0]
#         else:
#             c = cmap(clist[s]/max(clist))

#         X_trial = np.dot(X[s,:,:],D.T)
#         ax.plot3D(X_trial[:,0],X_trial[:,1],X_trial[:,2],linestyle,c = c, linewidth = linewidth, alpha = alpha)

In [24]:
# from tools_lnd import make_FP_axs, get_model_params #get_filename, 
# def compare_fp_lesions(m,cluster_var,lesion_cluster,rule_set,color_by = 'stim',n_components = 3,fontsize=20,
#                        review = False, epoch_set = [], q_thresh = 1e-3,t = 0, ri_axs = 0, view_init = [-51,30], 
#                        epoch_ri=[]):
    
#     if review:
#         lf = 'lesion_fps_review'
#     else:
#         lf = 'lesion_fps'
        
#     lesion_folder = 'lesion_fps_hierarchical_ward_distance_opt_clust'
#     cmap = plt.get_cmap('hsv')
#     c_full = 'dodgerblue'#cmap(0.1)
#     c_lesion = 'r'#'brown'#cmap(.6)
#     h_all_byepoch, h_all_byrule, _, _ = make_h_all(m)
    
#     plt.rcParams.update({'font.size': fontsize})
#     tick_fontsize = fontsize*.75
#     label_fontsize = fontsize*1

#     for ri in range(len(task_list)):
#         rule = task_list[ri]
#         trial = gen_trials_from_model_dir(m,rule,noise_on = False)
#         B = np.shape(trial.y_loc)[1]
#         N = np.shape(trial.y_loc)[1]
#         trial_set = range(0,B,int(B/10))
        
#         if len(epoch_ri)==0:
#             epoch = epoch_set[ri_axs]
#         else:
#             epoch =  epoch_ri  
            
#         rule = rule_set[ri_axs]
#         trial = gen_trials_from_model_dir(m,rule,noise_on = False)
#         filename,_ = get_filename(trial,epoch,t)
#         f = os.path.join(m,'tf_fixed_pts_all_init',rule,filename+'.npz')    
#         D_use = make_FP_axs(f, m, rule, epoch, axs = 'pca_h', clust = 'False')
#         w_in, b_in, w_out, b_out = get_model_params(m)

# #         D_use = -D_use
#         if rule == 'delaygo':
#             D_use = -D_use
#         else:
#             D_use[:,1] = -D_use[:,1]
            
#         D_use[:,2] = w_out[:,2]
#         axes_labels = [epoch, rule]

#         epoch_list = sorted(trial.epochs.items(), key=lambda x: x[1])
        
#         if len(epoch_set)==0:
#             epoch_set = [epoch_list[x][0] for x in range(len(epoch_list))]

#         for trial_num in [t,]:##range(0,B,int(B/5))
#             out_theta = int(180*trial.y_loc[-1,trial_num]/np.pi)

#             for plot_num in range(0,len(epoch_set)):
                
#                 fig = plt.figure(figsize=(7,7),tight_layout=True,facecolor='white')

#                 epoch = epoch_set[plot_num]
#                 epoch_name, task_name, _, _ = take_names(epoch,rule)

#                 ax = plt.subplot(1,1,1, projection='3d')
#                 w_color = .005
#                 ax.w_xaxis.set_pane_color((w_color, w_color, w_color, w_color))
#                 ax.w_yaxis.set_pane_color((w_color, w_color, w_color, w_color))
#                 ax.w_zaxis.set_pane_color((w_color, w_color, w_color, w_color))

#                 lesion_units_list = cluster_var['lesion_units_list'][lesion_cluster]
                
#                 if epoch[0]=='s':
#                     filename,_ = get_filename_fp(trial,epoch,trial_num)
#                 elif rule[0]=='d':
#                     filename,_ = get_filename_fp(trial,epoch,0)
#                 else:
#                     filename,_ = get_filename_fp(trial,epoch,trial_num)
                    
#                 f = os.path.join(m,lesion_folder,lf,'tf_fixed_pts_lesion'+str(lesion_cluster),rule,filename)
#                 c = c_lesion
                
#                 plot_FP_jitter_3D_lesion(lesion_units_list,ax,m,D_use,rule,trial_num,epoch,f,
#                                          rand_step_coef = 0,
#                                          al = .2, 
#                                          linestyle = '-', 
#                                          n_steps = 60,
#                                          n_jit = 0
#                                          ,c = c, 
#                                          q_thresh = q_thresh)

#                 lesion_units_list = []
                
#                 if epoch[0]=='s':
#                     filename,_ = get_filename_fp(trial,epoch,trial_num)
#                 elif rule[0]=='d':
#                     filename,_ = get_filename_fp(trial,epoch,0)
#                 else:
#                     filename,_ = get_filename_fp(trial,epoch,trial_num)
                    
#                 f = os.path.join(m,lesion_folder,lf,'tf_fixed_pts_lesion'+str(0),rule,filename)
#                 c = c_full

#                 plot_FP_jitter_3D_lesion(lesion_units_list,ax,m,D_use,rule,trial_num,epoch,f,
#                                          rand_step_coef = 0,
#                                          al = .2, 
#                                          linestyle = '-', 
#                                          n_steps = 60,
#                                          n_jit = 0,
#                                          c = c, 
#                                          q_thresh = q_thresh)

#                 tit = 'Task : '+ r"$\bf{" + task_name + "}$" +'\n Period : '+ r"$\bf{" + epoch_name.capitalize() + "}$"
#                 plt.title(tit,fontsize = fontsize,y = .9)

#                 ax.set_xlim([-1.2,1.2])
#                 ax.set_ylim([-1.2,1.2]) 
#                 make_ax_nice(ax,axes_labels)
                
#                 ax.set_yticks([])
#                 ax.set_xticks([])
#                 ax.set_zticks([])
                
#                 ax.view_init(azim=view_init[0], elev=view_init[1]) #(azim=-51, elev=30)
#                 ax.dist = 12
            

#                 figname = 'lesion'+str(lesion_cluster)+'_'+rule+'_'+epoch+'_'+str(out_theta)
#                 if not os.path.exists(os.path.join(figpath,'vis_fp_lesion')):
#                     os.makedirs(os.path.join(figpath,'vis_fp_lesion'))
#                 plt.savefig(os.path.join(figpath,'vis_fp_lesion',figname+'.pdf'))
#                 plt.show()

In [25]:
# def plot_FP_jitter_3D_lesion(lesion_units_list,ax,m,D_use,rule,t_num,fp_epoch,f,
#                    xlabel = 'FP set PC1',ylabel = 'FP set PC2',rand_step_coef = 0.1,n_steps = 100,
#                    lw = 3,al = .6,linestyle = '-',n_jit = 0,c = 'k', q_thresh = 1e-3):#, q_thresh = 1e-8

#     ms = 20
#     lw = 6
    
#     fp_struct = np.load(f)
#     step_fps = fp_struct['xstar']
    
# #     if fp_epoch=='stim1':
# #         fp_use = np.where(fp_struct['qstar']<q_thresh/100)[0] 
# #     else:
# #         fp_use = np.where(fp_struct['qstar']<q_thresh)[0]
        
#     fp_use = np.where(fp_struct['qstar']<q_thresh)[0]
    
    
#     sorted_fps = fp_struct['xstar'][fp_use,:]
  
#     fp_project = np.dot(sorted_fps,D_use)

#     # runs jitters around fps
#     stable = np.empty(len(sorted_fps))
#     for fp_ind in range(len(sorted_fps)):

#         dst_scale = 300

#         facecolors_3d = c
#         facecolors_2d = c
#         edgecolors = 'w'

#         for jit in range(n_jit):
#             h0 = sorted_fps[fp_ind,:] + rand_step_coef*npr.randn(N)
#             h_t = vanilla_run_with_h0(params, x_t, h0, hp)
#             jitter = np.dot(h_t,D_use)
#             ax.plot3D(jitter[:,0],jitter[:,1],jitter[:,2],'-',c = 'k',linewidth = 3)


#         if fp_epoch=='stim1':
#             ax.scatter(fp_project[fp_ind,0],fp_project[fp_ind,1],fp_project[fp_ind,2],'o',
#                       s = dst_scale, edgecolors = edgecolors, facecolors = facecolors_3d, alpha = al)
#         else:
#             ax.scatter(fp_project[fp_ind,0],fp_project[fp_ind,1],fp_project[fp_ind,2],'o',
#                       s = dst_scale, edgecolors = edgecolors, facecolors = facecolors_3d, alpha = al)
        
#     model = Model(m)
#     with tf.Session() as sess:
#         model.restore()
#         model._sigma=0
#         if len(lesion_units_list)>0:
#             model.lesion_units(sess, lesion_units_list)
#         hp = model.hp
#         alpha = hp['dt']/hp['tau']
#         var_list = model.var_list
#         params = [sess.run(var) for var in var_list]

#         trial = generate_trials(rule, hp, mode='test',noise_on=False)
#         feed_dict = tools.gen_feed_dict(model, trial, hp)
#         h_tf, _ = sess.run([model.h, model.y_hat], feed_dict=feed_dict) #(n_time, n_condition, n_neuron)
#         T,S,N = np.shape(h_tf)
#         T_inds = get_T_inds(trial,fp_epoch) # grab epoch time indexing
#         x_t = np.matlib.repmat(trial.x[T_inds[1],t_num,:],n_steps,1)
            
            
#         #runs one state for n_steps starting from initial conditions
#         for jit in range(1):
#             h0 = h_tf[T_inds[0],t_num,:]
#             h_t = vanilla_run_with_h0(params, x_t, h0, hp)
#             jitter = np.dot(h_t,D_use)
#             ax.plot3D(jitter[:,0],jitter[:,1],jitter[:,2],'-',c = c,alpha = .5, linewidth = lw)
#             ax.plot3D(jitter[:,0],jitter[:,1],jitter[:,2],'-k',linewidth = lw/2)
#             ax.plot3D(jitter[:1,0],jitter[:1,1],jitter[:1,2],'x',c = c,markersize = ms,markeredgewidth = 3)
#             ax.plot3D(jitter[-2:-1,0],jitter[-2:-1,1],jitter[-2:-1,2],'^',c = c,markersize = ms,markeredgewidth = 3)
#             ax.plot3D(jitter[:1,0],jitter[:1,1],jitter[:1,2],'xk',markersize = ms/2,markeredgewidth = 1)
#             ax.plot3D(jitter[-2:-1,0],jitter[-2:-1,1],jitter[-2:-1,2],'^k',markersize = ms/2,markeredgewidth = 1)
         
#     ax.dist = 12

In [26]:
# def lesion_endpt_diff(lesion_units_list,m,rule,n_steps = 100):#, q_thresh = 1e-8
        
#     model = Model(m)
#     with tf.Session() as sess:
#         model.restore()
#         model._sigma=0
#         if len(lesion_units_list)>0:
#             model.lesion_units(sess, lesion_units_list)
#         hp = model.hp
#         alpha = hp['dt']/hp['tau']
#         var_list = model.var_list
#         params = [sess.run(var) for var in var_list]

#         trial = generate_trials(rule, hp, mode='test',noise_on=False)
#         feed_dict = tools.gen_feed_dict(model, trial, hp)
#         h_lesion, _ = sess.run([model.h, model.y_hat], feed_dict=feed_dict) #(n_time, n_condition, n_neuron)

# #     model = Model(m)
# #     with tf.Session() as sess:
# #         model.restore()
# #         model._sigma=0
# #         hp = model.hp
# #         alpha = hp['dt']/hp['tau']
# #         var_list = model.var_list
# #         params = [sess.run(var) for var in var_list]

# #         trial = generate_trials(rule, hp, mode='test',noise_on=False)
# #         feed_dict = tools.gen_feed_dict(model, trial, hp)
# #         h_tf, _ = sess.run([model.h, model.y_hat], feed_dict=feed_dict) #(n_time, n_condition, n_neuron)
        
#     return h_lesion #, h_tf 

In [27]:
# def get_filename_fp(trial, epoch,t):
#     ind_stim_loc  = str(int(180*trial.y_loc[-1,t]/np.pi))
#     nonzero_stim = trial.stim_locs[0,:]<100
#     stim_names = '_'.join(str(int(180*x/np.pi)) for x in trial.stim_locs[t,nonzero_stim])
#     filename = epoch+'_trial'+str(t)+'_x'+stim_names+'_y'+ind_stim_loc+'.npz'

#     return filename, ind_stim_loc


# def get_filename(trial, epoch,t):
#     ind_stim_loc  = 180*trial.y_loc[-1,t]/np.pi
#     filename = epoch+'_'+str(round(ind_stim_loc,2))+'.npz'

#     return filename, ind_stim_loc

In [28]:
# def vanilla_run_with_h0(params, x_t, h0, hparams):

#     dt = hparams['dt']
#     tau = hparams['tau']
#     alpha = dt/tau
#     activation = hparams['activation']

#     h = h0
#     h_t = []
#     h_t.append(np.expand_dims(h0,axis=1))
#     for x in x_t:
#         h = rnn_vanilla(params, np.squeeze(h), np.squeeze(x.T), alpha, activation)
#         h_t.append(np.expand_dims(h,axis=1))

#     h_t = np.squeeze(np.array(h_t))  
#     return h_t

# def rnn_vanilla(params, h, x, alpha, activation):

#     if activation == 'softplus':
#         _activation = lambda x: np.log(np.exp(x) + 1)
#     elif activation == 'tanh':
#         _activation = lambda x: np.tanh(x)
#     elif activation == 'relu':
#         _activation = lambda x: x * (x > 0)
#     elif activation == 'power':
#         _activation = lambda x: (x * (x > 0))**2
#     elif activation == 'retanh':
#         _activation = lambda x: np.tanh(x * (x > 0))
    
#     xh = np.concatenate([x,h], axis=0)
#     gate_inputs = np.dot(params[0].T,xh)+params[1]
#     noise = 0
#     output = _activation(gate_inputs) # + noise

#     h_new = (1-alpha) * h + alpha * output
    
#     return h_new

In [29]:
# model = Model(m)
# with tf.Session() as sess:
#     model.restore()
#     model._sigma=0
#     if len(lesion_units_list)>0:
#         model.lesion_units(sess, lesion_units_list)
#     hp = model.hp
#     alpha = hp['dt']/hp['tau']
#     var_list = model.var_list
#     params = [sess.run(var) for var in var_list]

#     trial = generate_trials(rule, hp, mode='test',noise_on=False)

In [30]:
# trial = gen_trials_from_model_dir(m,'delaygo',noise_on = False)
# B = np.shape(trial.y_loc)[1]

In [6]:
from tools_lnd import find_closest_fp_loc

def append_fps(f, h_end, fps_all = [], q_thresh = 1e-2, t_set = []):
    
    if len(t_set)==0:
        t_set = range(0,80,2)
    
    fp_struct = np.load(f)
    fp_use = np.where(fp_struct['qstar']<q_thresh)[0]
    x_star = fp_struct['xstar'][fp_use,:]
    closest_fp, _ = find_closest_fp_loc(h_end,x_star)
    
    if trial_num==t_set[0]:
        fps_all = x_star[[closest_fp,],:]
    else:
        fps_all = np.append(fps_all,x_star[[closest_fp,],:],axis = 0) 

    return fps_all


def get_filename_fp(trial, epoch,t):
    ind_stim_loc  = 180*trial.y_loc[-1,t]/np.pi
    filename = epoch+'_'+str(round(ind_stim_loc,2))+'.npz'

    return filename, ind_stim_loc

w_in, b_in, w_out, b_out = get_model_params(m)

Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.
Instructions for updating:
Call initializer instance with the dtype argument instea

/Users/lauradriscoll/miniconda3/envs/flex_mult/lib/python3.10/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2024-03-29 02:05:06.842594: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/flexible_multitask/fig3/LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/flexible_multitask/fig3/LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1/model.ckpt


In [7]:
from tools_lnd import take_names
q_thresh = 1e-2
plot_h = 'h'
lf = 'lesion_fps_review'

plot3 = False
plot_zero_plane = False

########################
########################
########################
cluster_name_set = ['Continuous Memory','Category Memory']
rule_set = ['delaydm1','dmcgo']
use_pcs = 'prev_pc'
epoch_set =  ['delay1',]
# ########################
cluster_name_set = ['Anti Stimulus','Delayed Response']
rule_set = ['delaygo','delayanti']
use_pcs = 'pc'
epoch_set = ['stim1',]
# ########################
cluster_name_set = ['Modality1','Modality2']
rule_set = ['delaydm1','delaydm2']
use_pcs = 'prev_pc'
epoch_set = ['delay1',]
########################
########################
########################
epoch_set = ['go1',]
use_pcs = 'prev_pc'
plot3 = True
plot_zero_plane = True

for cluster_name in cluster_name_set:
    
    if cluster_name == 'Category Memory':
        lesion_cluster = 20
        lesion_units_list = []
        [lesion_units_list.append(x) for x in cluster_var['lesion_units_list'][20]] # 
        [lesion_units_list.append(x) for x in cluster_var['lesion_units_list'][21]] # 
    elif cluster_name == 'Continuous Memory':
        lesion_cluster = 1
        lesion_units_list = []
        [lesion_units_list.append(x) for x in cluster_var['lesion_units_list'][1]] # 
        [lesion_units_list.append(x) for x in cluster_var['lesion_units_list'][2]] # 
        
    else:
        lesion_cluster = cluster_number[cluster_name]+1
        lesion_units_list = cluster_var['lesion_units_list'][lesion_cluster]

    for rule in rule_set:
        
        trial = gen_trials_from_model_dir(m,rule,mode='test',noise_on = False)
        _,h_l = gen_X_from_model_dir(m,trial,lesion_units_list = lesion_units_list)
        _,h_full = gen_X_from_model_dir(m,trial)

        B = np.shape(trial.y_loc)[1]
        t_set = range(3,int(B),int(B/40))
#         t_set = range(int(B))

        for epoch in epoch_set: 

            T_inds = get_T_inds(trial,epoch) #get indices of epoch

            #if trials have diff inputs, concat fps from diff trials
            if np.any(np.diff(trial.x[T_inds[0],:,:],axis = 0)):
                skip_trials = 2
            else:
                skip_trials = int(B)

            #Make fp_set
            fps_l = []
            fps_all = []

            for trial_num in t_set:

#                 out_theta = int(180*trial.y_loc[-1,trial_num]/np.pi)
                
                if np.any(np.diff(trial.x[T_inds[0],:,:],axis = 0)):
                    filename,_ = get_filename_fp(trial,epoch,trial_num)
                else:
                    filename,_ = get_filename_fp(trial,epoch,0)

                f = os.path.join(m,lesion_folder,lf,'tf_fixed_pts_lesion'+str(lesion_cluster),rule,filename)
                fps_l = append_fps(f, h_l[:,trial_num,[T_inds[-1],]].T, fps_all = fps_l, 
                                   q_thresh = q_thresh, t_set = t_set)

                f = os.path.join(m,lesion_folder,lf,'tf_fixed_pts_lesion'+str(0),rule,filename)
                fps_all = append_fps(f, h_full[:,trial_num,[T_inds[-1],]].T, fps_all = fps_all, 
                                     q_thresh = q_thresh, t_set = t_set)

            #####do PCA on combined fps for viz axes
            if use_pcs=='fp':
                pca = PCA(n_components = 10)
                fps_pc = pca.fit_transform(np.append(fps_all,fps_l,axis = 0) )

                D_all = pca.components_
                x_label = 'combined FP PC1'
                y_label = 'combined FP PC2'
                axes_labels = [x_label,y_label]
            elif use_pcs=='out':
                D_all = w_out[:,1:].T

                x_label = 'output 1'
                y_label = 'output 2'
                axes_labels = [x_label,y_label]
            elif use_pcs=='prev_pc':
                pca = PCA(n_components = 10)
                h_cat = np.append(h_full[:,:,T_inds[0]-1],h_l[:,:,T_inds[0]-1],axis = 1).T
                fps_pc = pca.fit_transform(h_cat)

                D_all = pca.components_
                D_all[0,:] = -D_all[0,:]
                x_label = 'combined h0 PC1'
                y_label = 'combined h0 PC2'
                axes_labels = [x_label,y_label]
            else:
                pca = PCA(n_components = 10)
                h_cat = np.append(h_full[:,:,T_inds[-1]],h_l[:,:,T_inds[-1]],axis = 1).T
                fps_pc = pca.fit_transform(h_cat)

                D_all = pca.components_
                x_label = 'combined h_end PC1'
                y_label = 'combined h_end PC2'
                axes_labels = [x_label,y_label]
                
            
            #PLOT FPS
            if plot3:
                fig = plt.figure(figsize=(4,4),tight_layout=True,facecolor='white')
                ax = fig.add_axes([0,0,1,1], projection='3d');
            else:
                fig = plt.figure(figsize=(4,4),tight_layout=True,facecolor='white')
                ax = plt.subplot(1,1,1)
                
            al = .2

            if plot_h:
                for t in t_set:
                    
                    if plot3:

                        if plot_zero_plane:
                            D3 = np.concatenate((D_all[:2,:],w_out[:,1:2].T))
                        else:
                            D3 = np.concatenate((D_all[:3,:],w_out[:,1:2].T))
                            axes_labels.append(axes_labels[1][:-1]+'3')
                        
                        h = h_full[:,t,T_inds]
                        h_in_D = np.dot(D3,h)
                        ax.plot3D(h_in_D[0,:],h_in_D[1,:],h_in_D[2,:],'-', c = 'dodgerblue',linewidth = 3, alpha = al)
#                         ax.scatter(h_in_D[0,0],h_in_D[1,0],h_in_D[2,0],'x', c = 'dodgerblue',linewidth = 3, alpha = al, s = 10)
#                         ax.scatter(h_in_D[0,-1],h_in_D[1,-1],h_in_D[2,-1],'^', c = 'dodgerblue',linewidth = 3, alpha = al)

                        h = h_l[:,t,T_inds]
                        h_in_D = np.dot(D3,h)
                        ax.plot3D(h_in_D[0,:],h_in_D[1,:],h_in_D[2,:],'-', c = 'orangered',linewidth = 3, alpha = al)
#                         ax.scatter(h_in_D[0,0],h_in_D[1,0],h_in_D[2,0],'x', c = 'orangered',linewidth = 3, alpha = al, s = 10)
#                         ax.scatter(h_in_D[0,-1],h_in_D[1,-1],h_in_D[2,-1],'^', c = 'orangered',linewidth = 3, alpha = al)
                        
                    else:
                        h = h_full[:,t,T_inds]
                        h_in_D = np.dot(D_all,h)
                        plt.plot(h_in_D[0,:],h_in_D[1,:],'-', c = 'dodgerblue',linewidth = 3, alpha = al)
                        plt.plot(h_in_D[0,0],h_in_D[1,0],'x', c = 'dodgerblue',linewidth = 3, alpha = al, markersize = 10)
                        plt.plot(h_in_D[0,-1],h_in_D[1,-1],'^', c = 'dodgerblue',linewidth = 3, alpha = al, markersize = 10)

                        h = h_l[:,t,T_inds]
                        h_in_D = np.dot(D_all,h)
                        plt.plot(h_in_D[0,:],h_in_D[1,:],'-', c = 'orangered',linewidth = 3, alpha = al)
                        plt.plot(h_in_D[0,0],h_in_D[1,0],'x', c = 'orangered',linewidth = 3, alpha = al, markersize = 10)
                        plt.plot(h_in_D[0,-1],h_in_D[1,-1],'^', c = 'orangered',linewidth = 3, alpha = al, markersize = 10)

            
            
            if plot3:
                fp_in_D_all = np.dot(D3,fps_all.T)
                fp_in_D_l = np.dot(D3,fps_l.T)
                ax.scatter(fp_in_D_all[0,:],fp_in_D_all[1,:],fp_in_D_all[2,:],'o', c = 'dodgerblue',alpha = .8, s = 20)
                ax.scatter(fp_in_D_l[0,:],fp_in_D_l[1,:],fp_in_D_l[2,:],'o', c = 'orangered',alpha = .8, s = 20)
            else:
                fp_in_D_all = np.dot(D_all,fps_all.T)
                fp_in_D_l = np.dot(D_all,fps_l.T)
                plt.plot(fp_in_D_all[0,:],fp_in_D_all[1,:],'o', c = 'dodgerblue',alpha = .8)#, markersize = 10)
                plt.plot(fp_in_D_l[0,:],fp_in_D_l[1,:],'o', c = 'orangered',alpha = .8)#, markersize = 10)


            if plot3:
                if plot_zero_plane:

                    [x1,x2] = ax.get_xlim()
                    [y1,y2] = ax.get_ylim()

                    zlabel = 'out_null'
                    save_axes = zlabel
                    xx, yy = np.meshgrid(np.linspace(x1, x2, num=2), np.linspace(y1, y2, num=2))
                    z = xx*0
                    ax.plot_surface(xx, yy, z, alpha=0.1)
                    ax.set_zlabel(r"$\bf{Output}$"+r'$\cos{\theta}$',labelpad=-10, fontsize = 16)
#                 ax.text(x1, y1, 0, 'Output Null', (1,0,0))
                    ax.set_zlim([-.9,.9])
                
                else:
                    ax.set_zlabel(axes_labels[2],fontsize = 16)
                
                ax.set_xlabel(axes_labels[0],fontsize = 16)
                ax.set_ylabel(axes_labels[1],fontsize = 16)
                ax.set_zticks([])
                ax.set_xticks([])
                ax.set_yticks([])
                ax.dist = 12
                
                
            else:
                make_lil_axes(ax,axes_labels,fontsize = 16,fac_len = 1)
                
            epoch_name, rule_name, _, _ = take_names(epoch,rule,epoch_axes = epoch, h_epoch = epoch)
            tit = 'Lesion: '+ cluster_name + ' \n Task: ' + rule_name + ' \n Period: ' +epoch_name
            plt.title(tit,fontsize = 18,y = .9)

            #save figs
            if plot3:
                figname = '_'.join(('plot3',cluster_name,rule_name,epoch_name,x_label))
            else:
                figname = '_'.join((cluster_name,rule_name,epoch_name,x_label))
                
#             plt.tight_layout()    
            plt.savefig(os.path.join(figpath,figname+'.pdf'), bbox_inches='tight')

Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(148, 128) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(128, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/flexible_multitask/fig3/LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/flexible_multitask/fig3/LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1/model.ckpt
Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(148, 

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lauradriscoll/Documents/data/flexible_multitask/fig3/LeakyRNN_softplus_diag_15_tasks_128_n_rnn_lr7.0l2_w6.0_h6.0_fdgo_reactgo_delaygo_fdanti_reactanti_delayanti_delaydm1_delaydm2_contextdelaydm1_contextdelaydm2_multidelaydm_dmsgo_dmsnogo_dmcgo_dmcnogo_1/lesion_fps_hierarchical_ward_distance_opt_clust/lesion_fps_review/tf_fixed_pts_lesion7/delaydm1/go1_180.0.npz'